In [1]:
import pandas as pd
import streamlit as st
from datetime import timedelta
from multicall import Call
import plotly.express as px
import numpy as np


from mainnet_launch.data_fetching.get_state_by_block import (
    get_raw_state_by_blocks,
    get_state_by_one_block,
    identity_with_bool_success,
    safe_normalize_with_bool_success,
    build_blocks_to_use,
)
from mainnet_launch.destinations import DestinationDetails, get_destination_details, AutopoolConstants
from mainnet_launch.lens_contract import fetch_pools_and_destinations_df
from mainnet_launch.constants import (
    CACHE_TIME,
    AutopoolConstants,
    eth_client,
    ALL_AUTOPOOLS,
    AUTO_ETH,
    BAL_ETH,
    AUTO_LRT,
)


def _clean_summary_stats_info(success, summary_stats):
    if success is True:
        summary = {
            "destination": summary_stats[0],
            "baseApr": summary_stats[1] / 1e18,
            "feeApr": summary_stats[2] / 1e18,
            "incentiveApr": summary_stats[3] / 1e18,
            "safeTotalSupply": summary_stats[4] / 1e18,
            "priceReturn": summary_stats[5] / 1e18,
            "maxDiscount": summary_stats[6] / 1e18,
            "maxPremium": summary_stats[7] / 1e18,
            "ownedShares": summary_stats[8] / 1e18,
            "compositeReturn": summary_stats[9] / 1e18,
            "pricePerShare": summary_stats[10] / 1e18,
            # ignoring slashings costs, no longer part of model
        }
        return summary
    else:
        return None


def _build_summary_stats_call(
    autopool: AutopoolConstants,
    destination_vault_address: str,
    direction: str = "out",
    amount: int = 0,
) -> Call:
    # /// @notice Gets the safe price of the underlying LP token
    # /// @dev Price validated to be inside our tolerance against spot price. Will revert if outside.
    # /// @return price Value of 1 unit of the underlying LP token in terms of the base asset
    # function getValidatedSafePrice() external returns (uint256 price);
    # getDestinationSummaryStats uses getValidatedSafePrice. So when prices are outside tolerance this function reverts

    # TODO find a version of this function that won't revert,
    if direction == "in":
        direction_enum = 0
    elif direction == "out":
        direction_enum = 1
    return_types = "(address,uint256,uint256,uint256,uint256,int256,int256,int256,uint256,int256,uint256)"

    # cleaning_function = build_summary_stats_cleaning_function(autopool)
    return Call(
        autopool.autopool_eth_strategy_addr,
        [
            f"getDestinationSummaryStats(address,uint8,uint256)({return_types})",
            destination_vault_address,
            direction_enum,
            amount,
        ],
        [(destination_vault_address, _clean_summary_stats_info)],
    )


def _fetch_autopool_destination_data(autopool: AutopoolConstants) -> pd.DataFrame:
    destination_details = get_destination_details()
    blocks = build_blocks_to_use()

    calls = [
        _build_summary_stats_call(dest.autopool, dest.vaultAddress)
        for dest in destination_details
        if dest.autopool == autopool
    ]

    autopool_all_destinations_summary_stats_df = get_raw_state_by_blocks(calls, blocks)

    return autopool_all_destinations_summary_stats_df, destination_details


def _filter_and_format_summary_stats(
    autopool_all_destinations_summary_stats_df: pd.DataFrame,
    destination_details: list,
    pools_and_destinations_df: pd.DataFrame,
) -> pd.DataFrame:
    """Filters the summary stats DataFrame to only have data for the current destinations and formats it by destination vault name."""

    destination_addresses = [c for c in autopool_all_destinations_summary_stats_df.columns]
    destination_vault_address_to_destination = {d.vaultAddress: d for d in destination_details}

    def get_current_destinations_by_block(
        autopool: AutopoolConstants, getPoolsAndDestinations: pd.DataFrame
    ) -> list[str]:
        for a, list_of_destinations in zip(
            getPoolsAndDestinations["autopools"], getPoolsAndDestinations["destinations"]
        ):
            if a["poolAddress"].lower() == autopool.autopool_eth_addr.lower():
                return [dest["vaultAddress"] for dest in list_of_destinations]

    autopool_all_destinations_summary_stats_df["current_destinations"] = pools_and_destinations_df.apply(
        lambda row: get_current_destinations_by_block(AUTO_LRT, row["getPoolsAndDestinations"]), axis=1
    )

    def _limit_destination_summary_stats_to_current_destinations(row: dict):
        active_destinations = {}
        for addr in destination_addresses:
            destination_details = destination_vault_address_to_destination[addr]
            active_destinations[destination_details.vault_name] = row[addr]
        return active_destinations

    destination_name_to_destination_summary_stats_df = pd.DataFrame.from_records(
        autopool_all_destinations_summary_stats_df.apply(
            _limit_destination_summary_stats_to_current_destinations, axis=1
        )
    )
    destination_name_to_destination_summary_stats_df.index = autopool_all_destinations_summary_stats_df.index

    return destination_name_to_destination_summary_stats_df


def build_autopool_summary_stats_df(autopool: AutopoolConstants) -> pd.DataFrame:
    """Returns a DataFrame where the columns are the destination vault name, and the values are the dict from getDestinationSummaryStats()"""
    autopool_all_destinations_summary_stats_df, destination_details = _fetch_autopool_destination_data(autopool)
    pools_and_destinations_df = fetch_pools_and_destinations_df()
    destination_name_to_destination_summary_stats_df = _filter_and_format_summary_stats(
        autopool_all_destinations_summary_stats_df, destination_details, pools_and_destinations_df
    )

    return destination_name_to_destination_summary_stats_df


# def build_autopool_summary_stats_df(autopool: AutopoolConstants) -> pd.DataFrame:
#     """Returns a DataFrame where the columns are the destination vault name, and the values are the response from getDestinationSummaryStats()"""
#     destination_details = get_destination_details()
#     pools_and_destinations_df = fetch_pools_and_destinations_df()
#     blocks = build_blocks_to_use()
#     calls = [
#         _build_summary_stats_call(dest.autopool, dest.vaultAddress)
#         for dest in destination_details
#         if dest.autopool == autopool
#     ]

#     autopool_all_destinations_summary_stats_df = get_raw_state_by_blocks(calls, blocks)

#     destination_addresses = [c for c in autopool_all_destinations_summary_stats_df.columns]

#     def get_current_destinations_by_block(autopool: AutopoolConstants, getPoolsAndDestinations: pd.DataFrame) -> list[str]:
#         for a, list_of_destinations in zip(getPoolsAndDestinations["autopools"], getPoolsAndDestinations["destinations"]):
#             if a["poolAddress"].lower() == autopool.autopool_eth_addr.lower():
#                 # returns the currently active destinations for this autopool
#                 return [dest["vaultAddress"] for dest in list_of_destinations]


#     autopool_all_destinations_summary_stats_df["current_destinations"] = pools_and_destinations_df.apply(
#         lambda row: get_current_destinations_by_block(AUTO_LRT, row["getPoolsAndDestinations"]), axis=1
#     )

#     destination_vault_address_to_destination = {d.vaultAddress: d for d in destination_details}

#     def _limit_destination_summary_stats_to_current_destinations(row: dict):

#         active_destinations = {}

#         for addr in destination_addresses:
#             destination_details = destination_vault_address_to_destination[addr]
#             active_destinations[destination_details.vault_name] = row[addr]

#         return active_destinations

#     destination_name_to_destination_summary_stats_df = pd.DataFrame.from_records(
#         autopool_all_destinations_summary_stats_df.apply(
#             _limit_destination_summary_stats_to_current_destinations, axis=1
#         )
#     )
#     destination_name_to_destination_summary_stats_df.index = autopool_all_destinations_summary_stats_df.index
#     return destination_name_to_destination_summary_stats_df


autoETH_summary_stats = build_autopool_summary_stats_df(AUTO_LRT)
autoETH_summary_stats

2024-10-17 17:16:25.597 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-10-17 17:16:25.601 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-10-17 17:16:25.603 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-10-17 17:16:25.604 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-10-17 17:16:25.605 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-17 17:16:25.701 
  command:

    streamlit run /home/parker/Documents/Tokemak/v2-rebalance-dashboard/.venv/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-10-17 17:16:25.701 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-1

,Tokemak-Wrapped Ether-Balancer rsETH-WETH Stable Pool (balancer),Tokemak-Wrapped Ether-Balancer weETH/ezETH/rswETH (balancer),Tokemak-Wrapped Ether-Balancer weETH/rETH StablePool (balancer),Tokemak-Wrapped Ether-weETH/WETH-ng (curve),Tokemak autoLRT (Tokemak),Tokemak-Wrapped Ether-Balancer rsETH / ETHx (balancer),Tokemak-Wrapped Ether-Balancer ezETH-WETH Stable Pool (balancer)
timestamp,,,,,,,
2024-09-15 02:04:47,None,None,{'destination': '0x3b2c87ed95483059b950155a68e...,None,{'destination': '0xe800e3760fc20aa98c5df6a9816...,{'destination': '0xb6d68122428dc1141467cb96791...,None
2024-09-15 05:44:35,None,None,{'destination': '0x3b2c87ed95483059b950155a68e...,None,{'destination': '0xe800e3760fc20aa98c5df6a9816...,{'destination': '0xb6d68122428dc1141467cb96791...,None
2024-09-15 09:24:47,None,None,{'destination': '0x3b2c87ed95483059b950155a68e...,None,{'destination': '0xe800e3760fc20aa98c5df6a9816...,{'destination': '0xb6d68122428dc1141467cb96791...,None
2024-09-15 13:04:11,None,None,{'destination': '0x3b2c87ed95483059b950155a68e...,None,{'destination': '0xe800e3760fc20aa98c5df6a9816...,{'destination': '0xb6d68122428dc1141467cb96791...,None
2024-09-15 16:44:23,None,{'destination': '0x90300b02b162f902b9629963830...,{'destination': '0x3b2c87ed95483059b950155a68e...,{'destination': '0x777faf85c8e5fc6f4332e56b989...,{'destination': '0xe800e3760fc20aa98c5df6a9816...,{'destination': '0xb6d68122428dc1141467cb96791...,None
...,...,...,...,...,...,...,...
2024-10-17 08:01:11,{'destination': '0xf9779aef9f77e78c857cb4a068c...,{'destination': '0x90300b02b162f902b9629963830...,{'destination': '0x3b2c87ed95483059b950155a68e...,{'destination': '0x777faf85c8e5fc6f4332e56b989...,{'destination': '0xe800e3760fc20aa98c5df6a9816...,{'destination': '0xb6d68122428dc1141467cb96791...,{'destination': '0x2f2cc1bf461413014741dd68481...
2024-10-17 11:40:47,{'destination': '0xf9779aef9f77e78c857cb4a068c...,{'destination': '0x90300b02b162f902b9629963830...,{'destination': '0x3b2c87ed95483059b950155a68e...,{'destination': '0x777faf85c8e5fc6f4332e56b989...,{'destination': '0xe800e3760fc20aa98c5df6a9816...,{'destination': '0xb6d68122428dc1141467cb96791...,{'destination': '0x2f2cc1bf461413014741dd68481...
2024-10-17 15:21:11,{'destination': '0xf9779aef9f77e78c857cb4a068c...,{'destination': '0x90300b02b162f902b9629963830...,{'destination': '0x3b2c87ed95483059b950155a68e...,{'destination': '0x777faf85c8e5fc6f4332e56b989...,{'destination': '0xe800e3760fc20aa98c5df6a9816...,{'destination': '0xb6d68122428dc1141467cb96791...,{'destination': '0x2f2cc1bf461413014741dd68481...


In [3]:
compositeReturn_out_df = autoETH_summary_stats.map(
    lambda row: row["priceReturn"] if isinstance(row, dict) else None
).astype(float)

px.line(compositeReturn_out_df)

In [5]:
autoETH_summary_stats.values[-1][0]

{'destination': '0xf9779aef9f77e78c857cb4a068c65ccbee25baac',
 'baseApr': 0.0196821087299046,
 'feeApr': 0.002059510749280563,
 'incentiveApr': 0.06320866588379175,
 'safeTotalSupply': 3107.5585939492885,
 'priceReturn': 0.000423177979394055,
 'maxDiscount': 0.000912399533874503,
 'maxPremium': 0.0,
 'ownedShares': 844.9280278590563,
 'compositeReturn': 0.07905259675399179,
 'pricePerShare': 1.016326271270958}